In [17]:
# Title: Adjust Illumination for Folders of Images
# Include with dissertation
# Restored from corrupted version 
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil

def adjust_illumination(image, brightness=0, contrast=0, hue=0, saturation=0):
    """
    Adjusts the illumination properties of an image: brightness, contrast, hue, and saturation.

    Args:
        image (ndarray): The input image.
        brightness (int): Brightness adjustment (-100 to 100).
        contrast (int): Contrast adjustment (-100 to 100).
        hue (int): Hue adjustment (-100 to 100).
        saturation (int): Saturation adjustment (-100 to 100).

    Returns:
        ndarray: The adjusted image.
    """
    # Adjust brightness and contrast
    adjusted = cv2.convertScaleAbs(image, alpha=1 + contrast / 100.0, beta=brightness)

    # Convert to HSV to adjust hue and saturation
    hsv = cv2.cvtColor(adjusted, cv2.COLOR_BGR2HSV).astype(np.float32)

    # Adjust hue and saturation
    h, s, v = cv2.split(hsv)
    h = (h + hue) % 180  # Hue adjustment
    s = np.clip(s * (1 + saturation / 100.0), 0, 255)  # Saturation adjustment
    hsv_adjusted = cv2.merge([h, s, v]).astype(np.uint8)

    # Convert back to BGR
    final_image = cv2.cvtColor(hsv_adjusted, cv2.COLOR_HSV2BGR)
    return final_image

def change_illumination_for_folder(input_folder, brightness=0, contrast=0, hue=0, saturation=0):
    """
    Augments all images in a folder, saves augmented images into a specified output folder,
    and copies corresponding annotations with updated names.

    Args:
        input_folder (str): Path to the folder containing images and annotations.
        brightness (int): Brightness adjustment (-100 to 100).
        contrast (int): Contrast adjustment (-100 to 100).
        hue (int): Hue adjustment (-100 to 100).
        saturation (int): Saturation adjustment (-100 to 100).
    """
    job_name = os.path.basename(input_folder)
    print(f"Starting illumination adjustment for {job_name}")

    # Define output folder
    parent_folder = os.path.dirname(input_folder)
    augmented_folder = os.path.join(parent_folder, f"{job_name}_ILLUM")

    # Create the output folder if it doesn't exist
    if not os.path.exists(augmented_folder):
        os.makedirs(augmented_folder)

    images = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    annotations = [f for f in os.listdir(input_folder) if f.lower().endswith('.txt')]

    for img_name in images:
        input_image_path = os.path.join(input_folder, img_name)
        base_name = os.path.splitext(img_name)[0]
        
        #output_image_path = os.path.join(augmented_folder, f"{base_name}_illum.jpg") ## Save JPEGs instead 
        input_extension = os.path.splitext(img_name)[-1]  # Get original file extension
        output_image_path = os.path.join(augmented_folder, f"{base_name}_illum{input_extension}")

        # Read and augment the image
        image = cv2.imread(input_image_path)
        if image is None:
            print(f"Warning: Could not read image {input_image_path}. Skipping.")
            continue

        augmented_image = adjust_illumination(image, brightness, contrast, hue, saturation)

        # Save the augmented image
        cv2.imwrite(output_image_path, augmented_image)

        # Copy and rename the corresponding annotation file if it exists
        annotation_name = f"{base_name}.txt"
        input_annotation_path = os.path.join(input_folder, annotation_name)
        output_annotation_path = os.path.join(augmented_folder, f"{base_name}_illum.txt")

        if os.path.exists(input_annotation_path):
            shutil.copy(input_annotation_path, output_annotation_path)

    print(f"Illumination change completed for {job_name}. Augmented images and annotations saved to {augmented_folder}.")

def display_image_sample(input_folder):
    """
    Displays the first image from each unaugmented folder and their augmented counterparts.

    Args:
        input_folder (str): Path to the folder containing images.
    """
    unaugmented_folders = [f.path for f in os.scandir(input_folder) if f.is_dir() and not f.name.endswith('_ILLUM')]
    augmented_folders = {os.path.basename(f): f.path for f in os.scandir(input_folder) if f.is_dir() and f.name.endswith('_ILLUM')}

    num_folders = len(unaugmented_folders)
    rows = (num_folders + 3) // 4 * 2
    fig, axes = plt.subplots(rows, 4, figsize=(20, rows * 5))
    axes = axes.flatten()

    idx = 0

    for folder in unaugmented_folders:
        if idx >= len(axes):
            break
        folder_name = os.path.basename(folder)
        images = sorted([f for f in os.listdir(folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

        if images:
            image_path = os.path.join(folder, images[0])
            image = cv2.imread(image_path)
            axes[idx].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            axes[idx].set_title(f"Unaug: {folder_name}")
        else:
            axes[idx].set_title("No Image")
        axes[idx].axis("off")
        idx += 1

        if folder_name + "_ILLUM" in augmented_folders:
            aug_folder = augmented_folders[folder_name + "_ILLUM"]
            aug_images = sorted([f for f in os.listdir(aug_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

            if aug_images:
                aug_image_path = os.path.join(aug_folder, aug_images[0])
                aug_image = cv2.imread(aug_image_path)
                axes[idx].imshow(cv2.cvtColor(aug_image, cv2.COLOR_BGR2RGB))
                axes[idx].set_title(f"Aug: {folder_name}_ILLUM")
            else:
                axes[idx].set_title("No Aug Image")
        else:
            axes[idx].set_title("No Aug Folder")
        axes[idx].axis("off")
        idx += 1

    for ax in axes[idx:]:
        ax.axis("off")

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    # Adjust illumination settings
    # brightness = 60  # Increase brightness moderately
    # contrast = 20    # Enhance contrast slightly to bring out details
    # hue = 0          # Keep hue adjustment at 0 since the colors are natural
    # saturation = 30  # Increase saturation to enhance the vibrancy of the colors

    # output_folder = 'D:\\FlagDetectionDatasets\\ExportedDatasetsSelected\\Augmentation_llumination\\output'
    # change_illumination_for_folder_of_subfolders(input_folder, output_folder, brightness, contrast, hue, saturation)
    # change_illumination_for_folder_of_subfolders(input_folder, output_folder, brightness=60, contrast=20, hue=0, saturation=30)

    # Do instead for folders - results are better when tuned for a set

    # for job 54
    # Brightness: Increase by 40–60% to make the image less dim and highlight the flag 
    # Contrast: Increase by 30–50% to enhance the distinction between the flag and background.
    # Saturation: Increase by 30–50% to make the red and yellow colors of the flag more vibrant against the muted background.
    # Hue: Minimal adjustment (e.g., 0–10) to preserve the natural colors of the flag and sky.
    # adjusted_image = adjust_illumination(image, brightness=50, contrast=40, hue=0, saturation=40)

    # Stage 3: Run to fill the red yellow flag area with background - based on HSV
    #input_folder = 'D:/FlagDetectionDatasets/ExportedDatasetsReduced/Job_51_Stage_2'
    #change_illumination_for_folder(input_folder, brightness=50, contrast=30, hue=0, saturation=20)
    
    #input_folder = 'D:/FlagDetectionDatasets/ExportedDatasetsReduced/Job_51'
    #change_illumination_for_folder(input_folder, brightness=50, contrast=40, hue=0, saturation=20)

    #input_folder = 'D:/FlagDetectionDatasets/ExportedDatasetsReduced/Job_52'
    #change_illumination_for_folder(input_folder, brightness=50, contrast=40, hue=0, saturation=20)
    
    # Examples for other folders
    # input_folder = 'D:\\FlagDetectionDatasets\\ExportedDatasetsSelected\\Augmentation_llumination\\Job_54'
    # change_illumination_for_folder(input_folder, brightness=50, contrast=40, hue=0, saturation=40)
    
    # input_folder = 'D:\\FlagDetectionDatasets\\ExportedDatasetsSelected\\Augmentation_llumination\\Job_54_Aug'
    # change_illumination_for_folder(input_folder, brightness=50, contrast=40, hue=0, saturation=40)

    # input_folder = 'D:\\FlagDetectionDatasets\\ExportedDatasetsSelected\\Augmentation_llumination\\Job_7'
    # change_illumination_for_folder(input_folder, brightness=25, contrast=20, hue=0, saturation=15)
    # input_folder = 'D:\\FlagDetectionDatasets\\ExportedDatasetsSelected\\Augmentation_llumination\\Job_7_Aug'
    # change_illumination_for_folder(input_folder, brightness=25, contrast=20, hue=0, saturation=15)

    # input_folder = 'D:\\FlagDetectionDatasets\\ExportedDatasetsSelected\\Augmentation_llumination\\Job_55'
    # change_illumination_for_folder(input_folder, brightness=60, contrast=40, hue=0, saturation=25)
    # input_folder = 'D:\\FlagDetectionDatasets\\ExportedDatasetsSelected\\Augmentation_llumination\\Job_55_Aug'
    # change_illumination_for_folder(input_folder, brightness=60, contrast=40, hue=0, saturation=25)

    input_folder = 'D:/FlagDetectionDatasets/ExportedDatasetsReduced\Job_37'
    change_illumination_for_folder(input_folder, brightness=40, contrast=25, hue=0, saturation=30)
    # input_folder = 'D:\\FlagDetectionDatasets\\ExportedDatasetsSelected\\Augmentation_llumination\\Job_37_Aug'
    # change_illumination_for_folder(input_folder, brightnes

    #input_folder = 'D:/FlagDetectionDatasets/ExportedDatasetsReduced/Job_51_Stage_2'
    #change_illumination_for_folder(input_folder, brightness=50, contrast=30, hue=0, saturation=20)

    #input_folder = 'D:/FlagDetectionDatasets/Augmentation/Switch_class_in_images_sets/Switch_class_1_from_43_into_88/switch_flag_out_of_43'
    #change_illumination_for_folder(input_folder, brightness=25, contrast=20, hue=0, saturation=15)

    #display_image_sample('D:/FlagDetectionDatasets/ExportedDatasetsReduced')

    # Stage 3: Run to fill the red yellow flag area with background - based on HSV
    #input_folder = 'D:/FlagDetectionDatasets/ExportedDatasetsReduced/Job_52'
    #change_illumination_for_folder(input_folder, brightness=50, contrast=30, hue=0, saturation=20)

    # Display sample images for the input folder
    #displayImageSample('D:\\FlagDetectionDatasets\\ExportedDatasetsSelected\\Augmentation_llumination')



Starting illumination adjustment for obj_train_data


KeyboardInterrupt: 